<a href="https://colab.research.google.com/github/opeyemijanet/TECH_4_DEV_PROJECT/blob/main/Cash_flow_prediction_oop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import pandas as pd
from datetime import datetime


class CashflowPredictor:


    # Class-level constants
    CRITICAL_THRESHOLD = 30
    WARNING_THRESHOLD = 60
    MIN_DAYS_FOR_FULL_CONFIDENCE = 90
    MID_DAYS_FOR_MEDIUM_CONFIDENCE = 60
    MIN_DAYS_FOR_LOW_CONFIDENCE = 30
    MIN_TRANSACTIONS_PER_DAY = 5

    def __init__(self):
        """Initialize the predictor."""
        pass

    def predict(self, transactions: list, current_balance: float) -> dict:
        """
        Main prediction method. Analyses transaction history to determine risk.

        Parameters:
            transactions (list): List of transaction dicts with date, type, amount
            current_balance (float): Current cash balance

        Returns:
            dict: Prediction result with risk_level, days_until_broke, recommendations, etc.
        """
        # Input validation
        self._validate_inputs(transactions, current_balance)

        # Build DataFrame
        df = pd.DataFrame(transactions)

        # Calculate confidence
        confidence_score = self._calculate_confidence(df)

        # Separate income and expenses
        income_df = df[df["type"] == "income"]
        expense_df = df[df["type"] == "expense"]

        avg_income = income_df["amount"].mean() if not income_df.empty else 0.0
        avg_expense = expense_df["amount"].mean() if not expense_df.empty else 0.0

        burn_rate = avg_expense - avg_income

        # Classify risk
        risk_level, days_until_broke = self._classify_risk(burn_rate, current_balance)

        # Generate recommendations
        recommendations = self._generate_recommendations(risk_level)

        return {
            "risk_level": risk_level,
            "days_until_broke": days_until_broke,
            "avg_daily_income": round(avg_income, 2),
            "avg_daily_expense": round(avg_expense, 2),
            "burn_rate": round(burn_rate, 2),
            "confidence_score": confidence_score,
            "recommendations": recommendations,
            "created_at": datetime.now().isoformat(),
        }

    def _validate_inputs(self, transactions: list, current_balance: float) -> None:
        """Validate inputs before prediction."""
        if not transactions:
            raise ValueError("Transactions list cannot be empty.")

        if not isinstance(current_balance, (int, float)):
            raise TypeError(
                f"current_balance must be a number, got {type(current_balance).__name__}."
            )

        required_fields = {"date", "type", "amount"}
        for i, t in enumerate(transactions):
            missing = required_fields - set(t.keys())
            if missing:
                raise ValueError(
                    f"Transaction at index {i} is missing fields: {missing}"
                )

        # Validate transaction types
        df = pd.DataFrame(transactions)
        valid_types = {"income", "expense"}
        invalid = df[~df["type"].isin(valid_types)]
        if not invalid.empty:
            raise ValueError(
                f"Invalid transaction type(s) found: {invalid['type'].unique().tolist()}. "
                f"Must be one of: {valid_types}"
            )

    def _calculate_confidence(self, df: pd.DataFrame) -> float:
        """Calculate confidence score based on data quality."""
        df = df.copy()
        df['date'] = pd.to_datetime(df['date'])

        days_of_data = (df['date'].max() - df['date'].min()).days + 1

        if days_of_data >= self.MIN_DAYS_FOR_FULL_CONFIDENCE:
            days_score = 1.0
        elif days_of_data >= self.MID_DAYS_FOR_MEDIUM_CONFIDENCE:
            days_score = 0.85
        elif days_of_data >= self.MIN_DAYS_FOR_LOW_CONFIDENCE:
            days_score = 0.70
        else:
            days_score = 0.50

        transactions_per_day = len(df) / days_of_data
        density_score = 1.0 if transactions_per_day >= self.MIN_TRANSACTIONS_PER_DAY else 0.70

        return round(days_score * density_score, 2)

    def _classify_risk(self, burn_rate: float, current_balance: float) -> tuple:
        """Determine risk level and days until broke."""
        if burn_rate <= 0:
            return ("stable", None)

        days_until_broke = int(current_balance / burn_rate)

        if days_until_broke <= self.CRITICAL_THRESHOLD:
            risk_level = "critical"
        elif days_until_broke <= self.WARNING_THRESHOLD:
            risk_level = "warning"
        else:
            risk_level = "ok"

        return (risk_level, days_until_broke)

    def _generate_recommendations(self, risk_level: str) -> list:
        """Generate actionable recommendations based on risk."""
        recommendations_map = {
            "critical": [
                {"priority": 1, "action": "Reduce non-essential expenses immediately"},
                {"priority": 2, "action": "Follow up on all pending payments"},
                {"priority": 3, "action": "Consider short-term financing options"},
            ],
            "warning": [
                {"priority": 1, "action": "Review and cut discretionary spending"},
                {"priority": 2, "action": "Speed up collection of receivables"},
                {"priority": 3, "action": "Identify and plan new revenue sources"},
            ],
            "ok": [
                {"priority": 1, "action": "Monitor cashflow on a weekly basis"},
                {"priority": 2, "action": "Build an emergency cash reserve"},
            ],
            "stable": [
                {"priority": 1, "action": "Maintain current financial discipline"},
                {"priority": 2, "action": "Consider reinvesting surplus into inventory"},
            ],
        }

        return recommendations_map.get(risk_level, [
            {"priority": 1, "action": "Review your transaction data for accuracy"}
        ])



